# Dog Breed Identifier
All datasets are from the Kaggle dog-breed-identification challenge, courtesy Stanford. 

This notebook covers the logic behind training the model that powers this app.

## Imports and Libraries

In [1]:
%matplotlib inline
from __future__ import print_function

import keras
from keras.models import Sequential, Model
from keras.optimizers import Adam, SGD
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import ResNet50
from keras.applications.xception import Xception
from keras.layers import Dense, Flatten, Dropout
from keras.layers import GlobalAveragePooling2D, BatchNormalization, Dense
from keras.optimizers import Adam, SGD, RMSprop

Using TensorFlow backend.


## Hyperparameters

In [2]:
batch_size = 32
target_size = 229
epochs = 50

## Path to Data & Batches

Training data split into individual directories in order to use Keras' flow_from_directory method.
Roughly 10% of the Kaggle data was used for the validation set. I could have set this split percentage dynamically but after running a multitude of iterations with different split percentages, I settled on 10% (more on this when I have my own hardware)

See structure.py and split_data.py to see how I went about organizing the data for this model.

In [3]:
train_path = 'data/train'
valid_path = 'data/valid'

train_gen = ImageDataGenerator(rescale=1./ 255,
                               rotation_range=45,
                               width_shift_range=0.2,
                               height_shift_range=0.2,
                               shear_range=0.2, zoom_range=0.2,
                               horizontal_flip=True)

valid_gen = ImageDataGenerator(rescale=1./ 255)

train_batches = train_gen.flow_from_directory(train_path,
                                              target_size=(target_size, target_size),
                                              batch_size=batch_size,
                                              class_mode='categorical')

valid_batches = valid_gen.flow_from_directory(valid_path,
                                              target_size=(target_size, target_size),
                                              batch_size=batch_size,
                                              class_mode='categorical')

Found 9254 images belonging to 120 classes.
Found 968 images belonging to 120 classes.


## Transfer Learning

### Xception > ResNet50
I decided to go with xception for this iteration of the app. The top_layer has not been pulled in. Feel free to use ResNet50 but make sure to change the target_size to 224.

In [4]:
#base_model = ResNet50(weights='imagenet', include_top=False)
base_model = Xception(weights='imagenet', include_top=False, input_shape=(target_size, target_size, 3))

### Freeze Layers from base_model

In [5]:
for layer in base_model.layers:
    layer.trainable = False

### Fully Connected Layers

In [6]:
output = base_model.output
output = BatchNormalization()(output)
output = GlobalAveragePooling2D()(output)
output = Dropout(0.25)(output)
output = Dense(1024, activation='relu')(output)
output = Dropout(0.25)(output)
predictions = Dense(120, activation='softmax', name='predictions')(output)

In [7]:
#output = base_model.output
#output = GlobalAveragePooling2D()(output)
#output = Dropout(0.3)(output)
#output = Dense(1024, activation='relu')(output)
#predictions = Dense(120, activation='softmax', name='predictions')(output)

In [8]:
#output = base_model.get_layer(index=-1).output
model = Model(base_model.input, predictions)

### Optimizers

In [9]:
#optimizer = RMSprop(lr=0.001, rho=0.9)
optimizer = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=False)
#optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
#optimizer = 'rmsprop'

## Compile & Train

In [10]:
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit_generator(train_batches, steps_per_epoch=train_batches.n // batch_size, validation_data=valid_batches, validation_steps=valid_batches.n // batch_size, epochs=epochs)

Epoch 1/50
289/289 [==============================] - 116s 401ms/step - loss: 2.7443 - acc: 0.3680 - val_loss: 1.1399 - val_acc: 0.7427
Epoch 2/50
289/289 [==============================] - 115s 397ms/step - loss: 1.8304 - acc: 0.5304 - val_loss: 1.0897 - val_acc: 0.7618
Epoch 3/50
289/289 [==============================] - 114s 396ms/step - loss: 1.6869 - acc: 0.5547 - val_loss: 1.1585 - val_acc: 0.7585
Epoch 4/50
289/289 [==============================] - 114s 396ms/step - loss: 1.6530 - acc: 0.5621 - val_loss: 1.0805 - val_acc: 0.7671
Epoch 5/50
289/289 [==============================] - 114s 395ms/step - loss: 1.5586 - acc: 0.5843 - val_loss: 1.0035 - val_acc: 0.7767
Epoch 6/50
289/289 [==============================] - 114s 395ms/step - loss: 1.4900 - acc: 0.6002 - val_loss: 1.0624 - val_acc: 0.7596
Epoch 7/50
289/289 [==============================] - 115s 397ms/step - loss: 1.4560 - acc: 0.6049 - val_loss: 1.1781 - val_acc: 0.7532
Epoch 8/50
289/289 [============================

## Save Model

In [ ]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")
print("Saved model to disk")